<a href="https://colab.research.google.com/github/kbrezinski/JAX-Practice/blob/main/nn_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLP Training on MNIST

In [106]:
# init MLP and predict()
# torch dataloader
# training loop, loss fn
import jax
import numpy as np
import jax.numpy as jnp

from jax.scipy.special import logsumexp
from jax import jit, vmap, pmap, grad, value_and_grad

In [115]:
# constants
seed = 0
mnist_img_size = 784

def init_MLP(layer_widths, parent_key, scale=0.01):

  params = []
  keys = jax.random.split(parent_key, num=len(layer_widths) - 1)

  for in_width, out_width, key in zip(layer_widths[:-1], layer_widths[1:], keys):

    weight_key, bias_key = jax.random.split(key)
    params.append(
        [scale * jax.random.normal(weight_key, shape=(out_width, in_width)), # weights = (hidden dims, input)
         scale * jax.random.normal(bias_key, shape=(out_width,))]           # biases = (hidden_dims, 1)
    )
  return params

parent_key = jax.jax.random.PRNGKey(seed)
MLP_params = init_MLP([784, 512, 256, 10], parent_key)
jax.tree_map(lambda x: x.shape, MLP_params)

[[(512, 784), (512,)], [(256, 512), (256,)], [(10, 256), (10,)]]

In [121]:
def predict_MLP(params, x):

  hidden_layers = params[:-1]

  activation = x
  for w, b in hidden_layers:
    # ReLu( dot((512, 784)(784, 1)))
    activation = jax.nn.relu(jnp.dot(w, activation) + b)

  w_last, b_last = params[-1]
  logits = jnp.dot(w_last, activation) + b_last

  # mimics the same behavior as the softmax
  return logits - logsumexp(logits) # log(exp(o1)) - log(sum(exp(01), exp(02), ..., exp(03)))

# test on single image dimension (1, 784)
dummy_img_flat = np.random.randn(np.prod(mnist_img_size))
assert dummy_img_flat.shape == (784,)
prediction = predict_MLP(MLP_params, dummy_img_flat)
assert prediction.shape == (10,)

# test on a batched dimension (17, 784)
dummy_imgs_flats = np.random.randn(16, np.prod(mnist_img_size))
batched_MLP_predict = vmap(predict_MLP, in_axes=(None, 0))  # None to broadcast only params, batched dimension on zeroth dimension set to 0
predictions = batched_MLP_predict(MLP_params, dummy_imgs_flats)

In [117]:
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
## adding the dataloader

batch_size = 32
transform_ds = lambda x: np.ravel(np.array(x, dtype=np.float32) / 255.)

def collate_custom(batch):
  transposed_data = list(zip(*batch)) 
  return np.array(transposed_data[0]), np.array(transposed_data[1])

train_ds = MNIST(root='train_mnist', train=True, download=True, transform=transform_ds)
train_dl = DataLoader(train_ds, batch_size, shuffle=True, collate_fn=collate_custom)

test_ds = MNIST(root='test_mnist', train=False, download=True, transform=transform_ds)

In [119]:
## training loop function
epochs = 10

def loss_fn(params, imgs, gt_labels):
  # feedforward implementaiton
  predictions = batched_MLP_predict(params, imgs)  # size = (b, 10)
  return -jnp.mean(predictions * gt_labels)

def update(params, imgs, gt_labels, lr=0.01):
  # determine the gradients
  loss, grads = value_and_grad(loss_fn)(params, imgs, gt_labels)
  print(loss, grads)
  return loss, jax.tree_multimap(lambda p, g: p - lr * g, params, grads)

## main training loop
for epoch in range(epochs):

  for imgs, labels in train_dl:
    gt_labels = jax.nn.one_hot(labels, len(MNIST.classes))
    loss, MLP_params = update(MLP_params, imgs, labels)
    print(loss)

ValueError: ignored

In [ ]:
#